In [1]:
import geopandas as gpd
from tqdm.auto import tqdm

In [3]:
df = gpd.read_file("Data for testing/NZCCDv1.shp")
df

,Region,Site,Digitiser,Scale,Notes,Source,CPS,Proxy,Photoscale,Georef_ER,Pixel_Er,Total_UNCY,USDate,SHLength,Date,ID,geometry
0,Auckland,KarekareBethells,MW,2000,tod,RL,4,1,40000,5.03,1.416425,5.620681,01/02/2004,1.265403,2004-01-02,0.0,"LINESTRING Z (1728907.500 5916213.248 0.000, 1..."
1,Auckland,KarekareBethells,MW,2000,tod,RL,4,1,40000,5.03,1.416425,5.620681,01/02/2004,0.307010,2004-01-02,1.0,"LINESTRING Z (1729067.838 5914779.733 0.000, 1..."
2,Auckland,KarekareBethells,MW,2000,tod (landward),RL,4,1,40000,5.03,1.416425,5.620681,01/02/2004,0.255243,2004-01-02,2.0,"LINESTRING Z (1729647.881 5911983.994 0.000, 1..."
3,Auckland,KarekareBethells,MW,2000,tod,RL,3,1,40000,5.03,1.416425,5.314890,01/02/2004,1.107251,2004-01-02,3.0,MULTILINESTRING Z ((1728131.079 5917218.888 0....
4,Auckland,KarekareBethells,MW,2000,tod,RL,4,1,40000,5.03,1.416425,5.620681,01/02/2004,0.770375,2004-01-02,4.0,MULTILINESTRING Z ((1729643.228 5912906.682 0....
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19661,West Coast,WoodpeckerBay,MD,1000,EdgeVeg,RL,2,1,18800,4.00,0.630693,3.736680,11/18/1948,1.191368,1948-11-18,19661.0,"LINESTRING Z (1467237.671 5348414.193 0.000, 1..."
19662,Otago,Oamaru,TK,1500,Top of cliff,Max,3,2,0,2.02,0.412236,2.278429,04/04/2018,0.650952,2018-04-04,19662.0,"LINESTRING Z (1444386.944 5008247.367 0.000, 1..."
19663,Otago,Oamaru,TK,1500,Top of cliff,Max,3,2,0,2.02,0.412236,2.278429,04/04/2018,0.179434,2018-04-04,19663.0,"LINESTRING Z (1444860.629 5008686.925 0.000, 1..."
19664,Otago,Oamaru,TK,1500,Top of cliff,Max,3,2,0,2.02,0.412236,2.278429,04/04/2018,0.174825,2018-04-04,19664.0,"LINESTRING Z (1444996.286 5008822.252 0.000, 1..."


In [4]:
transects = gpd.read_file("Data for testing/Unique_ID_transects/NorthIsland_transects.shp")
transects

,Unique_ID,geometry
0,1.005226e+11,"LINESTRING Z (1680443.328 5669799.874 0.000, 1..."
1,1.005226e+11,"LINESTRING Z (1680451.296 5669793.831 0.000, 1..."
2,1.005226e+11,"LINESTRING Z (1680459.263 5669787.788 0.000, 1..."
3,1.005227e+11,"LINESTRING Z (1680467.271 5669781.803 0.000, 1..."
4,1.005227e+11,"LINESTRING Z (1680475.694 5669776.413 0.000, 1..."
...,...,...
139133,1.004662e+11,"LINESTRING Z (1799080.909 5395344.095 0.000, 1..."
139134,1.004662e+11,"LINESTRING Z (1799070.914 5395344.405 0.000, 1..."
139135,1.004662e+11,"LINESTRING Z (1799060.918 5395344.714 0.000, 1..."
139136,1.004662e+11,"LINESTRING Z (1799050.923 5395345.024 0.000, 1..."


In [5]:
points = gpd.read_file("Data for testing/Unique_ID_points/NorthIsland_UniqueID.shp")
points

,Unique_ID,geometry
0,1.003479e+11,POINT Z (1880322.836 5830555.229 0.000)
1,1.003479e+11,POINT Z (1880327.712 5830546.382 0.000)
2,1.003479e+11,POINT Z (1880333.202 5830537.997 0.000)
3,1.003479e+11,POINT Z (1880339.081 5830529.906 0.000)
4,1.003479e+11,POINT Z (1880345.252 5830522.035 0.000)
...,...,...
139133,1.004750e+11,POINT Z (1758086.752 5426055.275 0.000)
139134,1.004750e+11,POINT Z (1758091.623 5426064.015 0.000)
139135,1.004750e+11,POINT Z (1758096.813 5426072.563 0.000)
139136,1.004750e+11,POINT Z (1758102.158 5426081.017 0.000)
